<a href="https://colab.research.google.com/github/lacey-conrad/WGCNA-work/blob/main/Gene_coexp_GCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

import tensorflow


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU

# 1. Reading in the dataset:

In [3]:
df = pd.read_table('thermo_capacity_counts.txt')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_gastroc = pd.read_csv('gastroc_raw_counts.csv')

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
#/content/thermo_capacity_counts.txt

In [7]:
#df_copy = df.copy()

In [8]:
df_gastroc.head()

,Unnamed: 0,LN.F1.113.2be,LN.F1.113.aa,LN.F1.113.ae,LN.F1.113.ag,LN.F1.113.ak,LN.F1.113.am,LN.F1.113.b,LN.F1.113.h,LN.F1.113.p,...,ME.F1.145.ad,ME.F1.145.d,ME.F1.145.g,ME.F1.145.i,ME.F1.145.w,ME.F1.146.aa,ME.F1.146.ab,ME.F1.146.ac,ME.F1.146.be,ME.F1.146.bf
0,LOC121821266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,LOC102911281,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,LOC102926907,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LOC121821617,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,1,1,0
4,LOC102925994,3,4,3,4,7,7,9,2,6,...,3,9,5,6,11,2,2,4,2,5


In [9]:
df_gastroc['Row_Mean'] = df_gastroc.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [10]:
df_gastroc.head()

,Unnamed: 0,LN.F1.113.2be,LN.F1.113.aa,LN.F1.113.ae,LN.F1.113.ag,LN.F1.113.ak,LN.F1.113.am,LN.F1.113.b,LN.F1.113.h,LN.F1.113.p,...,ME.F1.145.d,ME.F1.145.g,ME.F1.145.i,ME.F1.145.w,ME.F1.146.aa,ME.F1.146.ab,ME.F1.146.ac,ME.F1.146.be,ME.F1.146.bf,Row_Mean
0,LOC121821266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,LOC102911281,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,LOC102926907,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,LOC121821617,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,1,1,0,0.404255
4,LOC102925994,3,4,3,4,7,7,9,2,6,...,9,5,6,11,2,2,4,2,5,4.319149


In [11]:
df_gastroc = df_gastroc[df_gastroc.Row_Mean >= 10]

In [12]:
df_gastroc.head()

,Unnamed: 0,LN.F1.113.2be,LN.F1.113.aa,LN.F1.113.ae,LN.F1.113.ag,LN.F1.113.ak,LN.F1.113.am,LN.F1.113.b,LN.F1.113.h,LN.F1.113.p,...,ME.F1.145.d,ME.F1.145.g,ME.F1.145.i,ME.F1.145.w,ME.F1.146.aa,ME.F1.146.ab,ME.F1.146.ac,ME.F1.146.be,ME.F1.146.bf,Row_Mean
32,LOC102903073,10,12,5,6,7,8,9,12,11,...,14,11,23,20,6,8,6,10,19,10.680851
42,LOC102904318,27,35,21,25,25,24,37,23,25,...,20,27,37,35,18,18,19,13,22,22.787234
48,Ppp2r1a,336,264,175,298,425,288,230,326,262,...,182,188,426,311,290,324,266,204,306,292.148936
55,LOC102906476,384,340,407,446,526,577,343,500,447,...,364,332,642,588,350,371,331,345,461,400.617021
60,LOC102922856,6,13,21,21,17,15,18,16,18,...,8,18,25,32,16,8,11,14,24,16.468085


In [13]:
df_gastroc = df_gastroc.T

In [14]:
df_gastroc.columns = df_gastroc.iloc[0]

In [15]:
df_gastroc = df_gastroc.iloc[1: , :]

In [16]:
df_gastroc.head()

Unnamed: 0,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,Cacng6,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
LN.F1.113.2be,10,27,336,384,6,13,33,83,23,111,...,11886,276,122,92,6169,5420,2144,16304,173,1511
LN.F1.113.aa,12,35,264,340,13,7,51,36,39,184,...,12638,371,207,81,7024,6495,1673,16561,142,1280
LN.F1.113.ae,5,21,175,407,21,10,43,43,30,200,...,9748,272,135,67,5841,5400,1414,13018,103,955
LN.F1.113.ag,6,25,298,446,21,16,54,61,23,131,...,16001,495,230,116,9873,7522,2011,21779,196,1599
LN.F1.113.ak,7,25,425,526,17,19,58,132,51,259,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [17]:
df_gastroc = df_gastroc.reset_index()

In [18]:
df_gastroc.head()

Unnamed: 0,index,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,LN.F1.113.2be,10,27,336,384,6,13,33,83,23,...,11886,276,122,92,6169,5420,2144,16304,173,1511
1,LN.F1.113.aa,12,35,264,340,13,7,51,36,39,...,12638,371,207,81,7024,6495,1673,16561,142,1280
2,LN.F1.113.ae,5,21,175,407,21,10,43,43,30,...,9748,272,135,67,5841,5400,1414,13018,103,955
3,LN.F1.113.ag,6,25,298,446,21,16,54,61,23,...,16001,495,230,116,9873,7522,2011,21779,196,1599
4,LN.F1.113.ak,7,25,425,526,17,19,58,132,51,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [19]:
df_gastroc = df_gastroc.rename(columns={'index': 'Treatment'})

In [20]:
df_gastroc.head()

Unnamed: 0,Treatment,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,LN.F1.113.2be,10,27,336,384,6,13,33,83,23,...,11886,276,122,92,6169,5420,2144,16304,173,1511
1,LN.F1.113.aa,12,35,264,340,13,7,51,36,39,...,12638,371,207,81,7024,6495,1673,16561,142,1280
2,LN.F1.113.ae,5,21,175,407,21,10,43,43,30,...,9748,272,135,67,5841,5400,1414,13018,103,955
3,LN.F1.113.ag,6,25,298,446,21,16,54,61,23,...,16001,495,230,116,9873,7522,2011,21779,196,1599
4,LN.F1.113.ak,7,25,425,526,17,19,58,132,51,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [21]:
df_gastroc['Treatment'] = df_gastroc['Treatment'].str.split('.').str[0]

In [22]:
df_gastroc.head()

Unnamed: 0,Treatment,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,LN,10,27,336,384,6,13,33,83,23,...,11886,276,122,92,6169,5420,2144,16304,173,1511
1,LN,12,35,264,340,13,7,51,36,39,...,12638,371,207,81,7024,6495,1673,16561,142,1280
2,LN,5,21,175,407,21,10,43,43,30,...,9748,272,135,67,5841,5400,1414,13018,103,955
3,LN,6,25,298,446,21,16,54,61,23,...,16001,495,230,116,9873,7522,2011,21779,196,1599
4,LN,7,25,425,526,17,19,58,132,51,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [23]:
cleanup_nums = {"Treatment": {"LN": 0, "ME": 1}}

In [24]:
df_gastroc = df_gastroc.replace(cleanup_nums)
df_gastroc.head()

Unnamed: 0,Treatment,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,0,10,27,336,384,6,13,33,83,23,...,11886,276,122,92,6169,5420,2144,16304,173,1511
1,0,12,35,264,340,13,7,51,36,39,...,12638,371,207,81,7024,6495,1673,16561,142,1280
2,0,5,21,175,407,21,10,43,43,30,...,9748,272,135,67,5841,5400,1414,13018,103,955
3,0,6,25,298,446,21,16,54,61,23,...,16001,495,230,116,9873,7522,2011,21779,196,1599
4,0,7,25,425,526,17,19,58,132,51,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [25]:
X = df_gastroc.drop(labels=['Treatment'], axis = 1)
y = df_gastroc.Treatment

In [26]:
genes = list(X.columns)

In [28]:
#genes

In [37]:
#df_gastroc_mat = X.to_numpy()

In [29]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [30]:
X = pd.DataFrame(X)

In [38]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422
0,-0.125081,0.579366,0.557992,-0.150439,-1.673688,0.366150,-0.622049,1.427171,-0.209314,-1.397398,...,0.272714,-0.080925,0.635755,0.408810,0.120288,-0.147767,0.157493,0.251173,0.757966,2.085768
1,0.242344,1.679576,-0.358187,-0.548785,-0.554494,-0.908593,0.737779,-0.753065,0.861463,0.272568,...,0.638972,0.373578,2.360018,0.182762,0.541458,0.794542,-0.630754,0.337486,0.389601,1.225683
2,-1.043642,-0.245792,-1.490686,0.057787,0.724584,-0.271222,0.133411,-0.428349,0.259151,0.638588,...,-0.768588,-0.100062,0.899466,-0.104935,-0.041284,-0.165298,-1.064206,-0.852430,-0.073825,0.015606
3,-0.859930,0.304313,0.074453,0.410867,0.724584,1.003521,0.964417,0.406635,-0.209314,-0.939873,...,2.276903,0.966823,2.826583,0.902006,1.944866,1.694775,-0.065091,2.089952,1.031268,2.413420
4,-0.676218,0.304313,1.690491,1.135132,0.085045,1.640893,1.266601,3.700183,1.664546,1.988286,...,0.932660,0.210914,1.812311,0.388261,0.910413,0.936545,-0.669246,0.388536,-0.002528,0.667186


In [39]:
df_gastroc_mat = X.to_numpy()

In [50]:
X = df_gastroc_mat.astype(int)

In [51]:
X_norm_genes = pd.DataFrame(data = df_gastroc_mat, columns = genes)

In [52]:
X_norm_genes.head()

,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,Cacng6,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,-0.125081,0.579366,0.557992,-0.150439,-1.673688,0.366150,-0.622049,1.427171,-0.209314,-1.397398,...,0.272714,-0.080925,0.635755,0.408810,0.120288,-0.147767,0.157493,0.251173,0.757966,2.085768
1,0.242344,1.679576,-0.358187,-0.548785,-0.554494,-0.908593,0.737779,-0.753065,0.861463,0.272568,...,0.638972,0.373578,2.360018,0.182762,0.541458,0.794542,-0.630754,0.337486,0.389601,1.225683
2,-1.043642,-0.245792,-1.490686,0.057787,0.724584,-0.271222,0.133411,-0.428349,0.259151,0.638588,...,-0.768588,-0.100062,0.899466,-0.104935,-0.041284,-0.165298,-1.064206,-0.852430,-0.073825,0.015606
3,-0.859930,0.304313,0.074453,0.410867,0.724584,1.003521,0.964417,0.406635,-0.209314,-0.939873,...,2.276903,0.966823,2.826583,0.902006,1.944866,1.694775,-0.065091,2.089952,1.031268,2.413420
4,-0.676218,0.304313,1.690491,1.135132,0.085045,1.640893,1.266601,3.700183,1.664546,1.988286,...,0.932660,0.210914,1.812311,0.388261,0.910413,0.936545,-0.669246,0.388536,-0.002528,0.667186


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_norm_genes, y, test_size=0.20, random_state = 42)

In [ ]:
#X.head()

In [ ]:
# Model#1
# Input layer: 32 nodes
# Hidden layer 1: 64 nodes
# Optimizer: Adam
# Loss: Binary Crossentropy
# Metrics: Accuracy
# Epochs: 10
# Batch size: 10

In [59]:
# Initializing the ANN
model = Sequential()

# Adding the input layers and hidden layers:
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))

# Adding the output layer:
model.add(Dense(1, activation = 'sigmoid'))

# Compiling the ANN:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])



In [61]:
# Training the ANN:
#model.fit(X_train, y_train.to_numpy(), epochs = 10, batch_size = 10, verbose=1)

In [ ]:
# Making predictions:
y_pred = model.predict_classes(X_test)
model.evaluate(X_test, y_test.to_numpy())

In [ ]:
# Predictions of the response variable (whether or not the customer will exit the bank)
y_pred

In [ ]:
# Evaluating the model:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# Model#2 - more nodes
# Input layer: 50 nodes
# Hidden layer 1: 500 nodes
# Optimizer: Adam
# Loss: Binary Crossentropy
# Metrics: Accuracy
# Epochs: 10
# Batch size: 10

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train.to_numpy(), epochs = 10, batch_size = 10, verbose=1) 

In [ ]:
y_pred = model.predict_classes(X_test)
model.evaluate(X_test, y_test.to_numpy())

In [ ]:
y_pred

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# Model#3 - larger batch size
# Input layer: 32 nodes
# Hidden layer 1: 64 nodes
# Optimizer: Adam
# Loss: Binary Crossentropy
# Metrics: Accuracy
# Epochs: 10
# Batch size: 500

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train.to_numpy(), epochs = 10, batch_size = 500, verbose=1) 

In [ ]:
y_pred = model.predict_classes(X_test)
model.evaluate(X_test, y_test.to_numpy())

In [ ]:
y_pred

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# Model#4 - More epochs
# Input layer: 32 nodes
# Hidden layer 1: 64 nodes
# Optimizer: Adam
# Loss: Binary Crossentropy
# Metrics: Accuracy
# Epochs: 100
# Batch size: 10

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train.to_numpy(), epochs = 100, batch_size = 10, verbose=1) 

In [ ]:
y_pred = model.predict_classes(X_test)
model.evaluate(X_test, y_test.to_numpy())

In [ ]:
y_pred

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

# Insights derived from altering model hyperparameters:

- A smaller batch size produced more accurate results, but took longer to run.

- Adding more nodes in the hidden layers did not seem to improve the accuracy of the model.

- Increasing the number of epochs decreased the accuracy.

- My best model was number one:
 
 Model#1
 
 Input layer: 32 nodes
 
 Hidden layer 1: 64 nodes
 
 Optimizer: Adam
 
 Loss: Binary Crossentropy
 
 Metrics: Accuracy
 
 Epochs: 10
 
 Batch size: 10
 
True negatives: 1514 (Those who my model predicted would not exit, who did not actually exit)

False negatives: 79 (Those who my model predicted would exit, but who did not actually exit)

False positive: 200 (Those who my model predicted would not exit, but who did actually exit)

True positives: 207 (Those who my model predicted would exit the bank, who did actually exit the bank)

 accuracy of model: 86%